In [37]:
# Лабораторная работа 3 по дисциплине МРЗвИС
# Выполнена студентом группы 121702
# БГУИР Заломов Роман Андреевич
#
# Вариант 15: Реализовать модель рекуррентной сети с цепью нейросетевых моделей управляемых рекуррентных блоков 
# с логарифмической функцией активации (гиперболический арксинус) выходного сигнала на скрытом слое
#
# 24.12.24
# Данный файл содержит код построения графиков различных зависимостей

In [38]:
import numpy as np
from gru import (
    GRUModel,
    fibonacci_generator,
    one_by_n_generator,
    plus_one_minus_one_generator,
    squared_generator,
    half_generator,
    arithmetic_progression,
    create_sliding_window_batches,
    one_half_generator,
    sin_generator,    
    mape
)
from gru_with_adam import GRUAdam
import plotly.graph_objects as go
from math import pi

In [39]:
def scale_sequence(sequence):
    mean = np.mean(sequence)
    deviation = np.std(sequence)    
    scaled = (np.array(sequence) - mean) / deviation    
    return scaled, mean, deviation

def descale_sequence(scaled, mean, deviation):
    descaled = scaled * deviation + mean
    return descaled

def predict_next_n_elements(sequence, window_size, batch_size, hidden_size, n,
                            max_epochs, verbosity, lr, use_adam: bool = False):
    scaled, mean, deviation = scale_sequence(sequence)        
    # scaled = sequence  
    X, y = create_sliding_window_batches(scaled, window_size, batch_size, n)
    if not use_adam:
        model = GRUModel(window_size, hidden_size, n)
    else:
        model = GRUAdam(window_size, hidden_size, n)
    model.train(X, y, lr, max_epochs, verbosity=verbosity)
    train_pred, _, _ = model.forward(X)
    training_mape = mape(y, train_pred)           
    x_last = np.vstack(tuple(scaled[-window_size:] for _ in range(batch_size)))
    x_last = np.expand_dims(x_last, axis=0)    
    pred, _, _ = model.forward(x_last)
    descaled_pred = descale_sequence(pred[-1][-1], mean, deviation)
    # descaled_pred = pred[-1][-1]
    return training_mape, model, descaled_pred, mean, deviation

In [40]:
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
epochs = list(range(100, 2100, 100))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_epochs: list[int] = []

seq_length = 20
window_size = 5
batch_size = 1
hidden_size = 6
output_size = 3

verbosity = 100000
lr = 1e-2
use_adam = False

sequence = list(arithmetic_progression(seq_length + output_size, 1, 1))

for max_epochs in epochs:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, window_size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)

        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)
            

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 100/100, Loss: 0.286522
MAPE: 114.374474%
TRAINING FINISHED
Epoch 100/100, Loss: 0.191074
MAPE: 97.845095%
TRAINING FINISHED
Epoch 100/100, Loss: 0.201103
MAPE: 81.613798%
TRAINING FINISHED
Epoch 100/100, Loss: 0.239291
MAPE: 97.315626%
TRAINING FINISHED
Epoch 100/100, Loss: 0.200409
MAPE: 103.953198%
TRAINING FINISHED
Epoch 200/200, Loss: 0.177511
MAPE: 90.659416%
TRAINING FINISHED
Epoch 200/200, Loss: 0.067001
MAPE: 57.770198%
TRAINING FINISHED
Epoch 200/200, Loss: 0.139088
MAPE: 70.856538%
TRAINING FINISHED
Epoch 200/200, Loss: 0.154058
MAPE: 100.214669%
TRAINING FINISHED
Epoch 200/200, Loss: 0.143562
MAPE: 81.273861%
TRAINING FINISHED
Epoch 300/300, Loss: 0.051633
MAPE: 46.816176%
TRAINING FINISHED
Epoch 300/300, Loss: 0.059098
MAPE: 64.503772%
TRAINING FINISHED
Epoch 300/300, Loss: 0.115552
MAPE: 78.533218%
TRAINING FINISHED
Epoch 300/300, Loss: 0.047254
MAPE: 40.282347%
TRAINING FINISHED
Epoch 300/300, Loss: 0.079795
MAPE: 67.333501%
TRAINING FINISHED
Epoc

In [41]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_epochs, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=epochs, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от количества итераций обучения',
    font=dict(size=20),
    xaxis=dict(title='Количество итераций обучения'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [42]:
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
hidden_sizes = list(range(2, 22, 2))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 20
window_size = 3
batch_size = 1
output_size = 3

verbosity = 100000
lr = 5e-3
max_epochs = 500
use_adam = False

sequence = list(arithmetic_progression(seq_length + output_size, 1, 1))

for size in hidden_sizes:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):
        all_sizes.append(size)        
        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, window_size, batch_size,
            size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)

        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 500/500, Loss: 0.313054
MAPE: 116.085258%
TRAINING FINISHED
Epoch 500/500, Loss: 0.189225
MAPE: 87.225613%
TRAINING FINISHED
Epoch 500/500, Loss: 0.271174
MAPE: 101.322652%
TRAINING FINISHED
Epoch 500/500, Loss: 0.157079
MAPE: 75.692446%
TRAINING FINISHED
Epoch 500/500, Loss: 0.224639
MAPE: 96.920583%
TRAINING FINISHED
Epoch 500/500, Loss: 0.102870
MAPE: 62.486577%
TRAINING FINISHED
Epoch 500/500, Loss: 0.245793
MAPE: 97.412380%
TRAINING FINISHED
Epoch 500/500, Loss: 0.198485
MAPE: 94.760749%
TRAINING FINISHED
Epoch 500/500, Loss: 0.168840
MAPE: 73.565108%
TRAINING FINISHED
Epoch 500/500, Loss: 0.083684
MAPE: 51.140527%
TRAINING FINISHED
Epoch 500/500, Loss: 0.192812
MAPE: 82.786189%
TRAINING FINISHED
Epoch 500/500, Loss: 0.126134
MAPE: 64.537909%
TRAINING FINISHED
Epoch 500/500, Loss: 0.133269
MAPE: 63.870042%
TRAINING FINISHED
Epoch 500/500, Loss: 0.152888
MAPE: 70.350054%
TRAINING FINISHED
Epoch 500/500, Loss: 0.099835
MAPE: 64.740451%
TRAINING FINISHED
Epoch

In [43]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=hidden_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>количества нейронов на скрытом слое',
    font=dict(size=20),
    xaxis=dict(title='Количество нейронов на скрытом слое'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [44]:
# MAPE relation with sliding window
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()


all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 40
window_sizes = list(range(2, seq_length // 2, 2))
hidden_size = 8
batch_size = 1
output_size = 3

verbosity = 100000
lr = 1e-2
max_epochs = 250
use_adam = False

sequence = list(arithmetic_progression(seq_length + output_size, 1, 1))

for size in window_sizes:        

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):
        all_sizes.append(size)              
        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)
        
        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 250/250, Loss: 0.045420
MAPE: 44.494578%
TRAINING FINISHED
Epoch 250/250, Loss: 0.111794
MAPE: 52.444295%
TRAINING FINISHED
Epoch 250/250, Loss: 0.042035
MAPE: 44.981416%
TRAINING FINISHED
Epoch 250/250, Loss: 0.032653
MAPE: 40.677738%
TRAINING FINISHED
Epoch 250/250, Loss: 0.051842
MAPE: 46.561302%
TRAINING FINISHED
Epoch 250/250, Loss: 0.031021
MAPE: 60.137445%
TRAINING FINISHED
Epoch 250/250, Loss: 0.045199
MAPE: 48.798401%
TRAINING FINISHED
Epoch 250/250, Loss: 0.039759
MAPE: 63.856625%
TRAINING FINISHED
Epoch 250/250, Loss: 0.024006
MAPE: 42.989415%
TRAINING FINISHED
Epoch 250/250, Loss: 0.066966
MAPE: 52.792896%
TRAINING FINISHED
Epoch 250/250, Loss: 0.020254
MAPE: 51.302164%
TRAINING FINISHED
Epoch 250/250, Loss: 0.018561
MAPE: 44.135124%
TRAINING FINISHED
Epoch 250/250, Loss: 0.022036
MAPE: 50.694265%
TRAINING FINISHED
Epoch 250/250, Loss: 0.021637
MAPE: 54.517646%
TRAINING FINISHED
Epoch 250/250, Loss: 0.038012
MAPE: 50.175741%
TRAINING FINISHED
Epoch 2

In [45]:
# Plotting results
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=window_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>размера скользящего окна',
    font=dict(size=20),
    xaxis=dict(title='Размер скользящего окна'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [46]:
# Настройки эксперимента
experiments_amount = 5
seq_length = 10
window_size = 3
batch_size = 1
hidden_size = 10
output_size = 3
max_epochs = 2000
verbosity = 100000
lr = 1e-2

# Словарь с генераторами
sequence_generators = {
    "1, 1, 2, 3, 5, 8,...": lambda n: list(fibonacci_generator(n)),
    "1, 4, 9, 16,...": lambda n: list(squared_generator(n)),
    "1, 2, 3, 4, 5,...": lambda n: list(arithmetic_progression(n, a0=1, d=1)),    
    "1, 0.5 ,0.(3), 0.25, 0.2,...": lambda n: list(one_by_n_generator(n)),
    "1, -1, 1, -1,...": lambda n: list(plus_one_minus_one_generator(n)),
    "1, 0.5, 1, 0.5, 0.5, 1,...": lambda n: list(one_half_generator(n)),
}

all_mapes = []
mapes_mean = []
all_sequences = []

# Тренировка для каждой последовательности
for seq_name, seq_generator in sequence_generators.items():
    sequence = seq_generator(seq_length + output_size)
    print(sequence)   

    mapes_sum = 0
    for _ in range(experiments_amount):
        all_sequences.append(seq_name)
        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, window_size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)
        
        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)

    mapes_mean.append(mapes_sum / experiments_amount)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.080018
MAPE: 112.208121%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.104843
MAPE: 125.815779%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.048081
MAPE: 86.437790%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.042514
MAPE: 77.859735%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.074972
MAPE: 111.674799%
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169]
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.009073
MAPE: 37.295664%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.007252
MAPE: 25.546749%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.006347
MAPE: 31.110579%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.004189
MAPE: 25.265090%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.004472
MAPE: 20.134286%
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.001374
MAPE: 12.136676%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.011811
MAPE: 35.280334%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.00304

In [47]:
# Построение графика
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sequences, y=all_mapes, name="Все наблюдения", mode="markers")
)
fig.add_trace(
    go.Scatter(x=list(sequence_generators.keys()), y=mapes_mean, name="Средние значения", mode="markers")
)
fig.update_layout(
    title="Зависимость средней абсолютной ошибки<br>от вида последовательности",
    font=dict(size=20),
    xaxis=dict(title="Вид последовательности"),
    yaxis=dict(title="Средняя абсолютная ошибка"),
    width=1280,
    height=720
)
fig.show()

In [48]:
# Arithmetic
seq_length = 20
window_size = 5
batch_size = 1
hidden_size = 10
output_size = 3

sequence = list(arithmetic_progression(seq_length, 1, 1))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")  

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.075597
MAPE: 2.774577%
TEST
Предсказания на тестовой выборке:
 [17.19924749 18.28864466 19.29795358]
Истинное значение на тестовой выборке:
 [18 19 20]
MAPE: 3.9009%


In [49]:
# Fibonacci
seq_length = 15
window_size = 5
batch_size = 1
hidden_size = 10
output_size = 3

sequence = list(fibonacci_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")

TRAINING FINISHED
Epoch 2000/2000, Loss: 14.883284
MAPE: 5.824914%
TEST
Предсказания на тестовой выборке:
 [141.2765023  228.73504274 370.11261634]
Истинное значение на тестовой выборке:
 [144 233 377]
MAPE: 1.8496%


In [50]:
# 1 / n
seq_length = 20
window_size = 5
batch_size = 1
hidden_size = 10
output_size = 3

sequence = list(one_by_n_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.001188
MAPE: 34.203959%
TEST
Предсказания на тестовой выборке:
 [0.0508449  0.05290368 0.03482916]
Истинное значение на тестовой выборке:
 [0.05555556 0.05263158 0.05      ]
MAPE: 13.1126%


In [51]:
# 1, -1, 1, -1, ...
seq_length = 20
window_size = 5
batch_size = 1
hidden_size = 10
output_size = 3

sequence = list(plus_one_minus_one_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 2000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%") 

Epoch 2000/2000, Loss: 0.243853
MAPE: 68.737718%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.243743
MAPE: 68.721470%
TEST
Предсказания на тестовой выборке:
 [-0.32862481  0.1850694  -0.46685774]
Истинное значение на тестовой выборке:
 [-1  1 -1]
MAPE: 67.3149%


In [52]:
# n ** 2
seq_length = 10
window_size = 3
batch_size = 1
hidden_size = 10
output_size = 3

sequence = list(squared_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%") 

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.110145
MAPE: 0.909499%
TEST
Предсказания на тестовой выборке:
 [64.01295662 80.92381294 99.97013976]
Истинное значение на тестовой выборке:
 [ 64  81 100]
MAPE: 0.0481%
